# Get What You Want, Not What You Don't: Image Content Suppression for Text-to-Image Diffusion Models-ICLR2024

图像内容编辑的内容大多是对attention进行编辑(P2P、MaskCtrl)等，本文从**text embedding**角度进行图像内容编辑，不涉及模型的训练和微调，只在推理阶段对text embeddings进行更新实现。

注意:这篇文章里的negative prompt不同于Comfyui中的negative prompt，而是在prompt中的不想要的东西

## Observation from expriment

- observation 1:[EOT]的embedding中包含输入prompt的全局信息



对于用户提示"A man without glasses"，生成图像中包含眼镜（第一列）；<p>
将"glasses"对应的embedding置0，眼镜仍然会表达（第二和第三列）；<p>
同样，将全部[EOT] embeddings置0，仍然会生成眼镜（第四和第五列）；<p>
最后，当同时将"glasses"和[EOT] embeddings置0时，图像中的眼镜会移除（第六和第七列）

![alt text](1.jpg)

`在文本到图像扩散模型中，用户提示prompt首先经过文本编码器编码为text embedings，然后和latent code一起输入模型进行图像生成。因为text embeddings的长度一般都是固定的（例如SD1.4中的长度为77,text embeddings中[EOT] embeddings的长度为76-|p|），第一个embedding为开始标志位（start-of-text），定义为[SOT] embedding，接着是prompt对应的|p|个embedding，|p|为prompt中单词个数，剩余的都用结束标志位（end-of-text）填充，定义为[EOT] embeddings。`

- observation 2:[EOT] embeddings具有低秩特性，并且包含的语义信息是冗余的

使用WNNM对[EOT] embeddings进行低秩约束，当约束[EOT] embeddings的秩为9时，生成图像的PSNR已经大于40，可以认为与原始生成图像一致
- WNNM:The weighted nuclear norm minimization is an effective low-rank analysis method

- observation 3:[EOT] embeddings是高度相关的

随机从[EOT] embeddings中挑选一个embedding，复制|p|次，然后替换prompt对应的embeddings（图2(c)左），进行多次挑选与替换，发现生成的图像都与原始prompt生成的图像具有相同的语义（图(2)右），这表明76-|p|个[EOT] embeddings是高度相关的；图2(c)中间的距离度量同样说明EOT] embeddings之间的高度相关性

将用户提示prompt中的negative target进行抑制<p>
同时将negative target信息从76-|p|个[EOT] embeddings中移除

![alt text](2.jpg)

## Method

### 软权重正则化（Soft-weighted Regularization）

- text embedding:$\boldsymbol{c}=\{\boldsymbol{c}^{SOT},\boldsymbol{c}_0^P,\cdots,\boldsymbol{c}_{|\boldsymbol{p}|-1}^P,\boldsymbol{c}_0^{EOT},\cdots,\boldsymbol{c}_{N-|\boldsymbol{p}|-2}^{EOT}\}\in\mathbb{R}^{M\times N}$<p>
    - 手动将$\boldsymbol{c}_{0}^{P}$分割为<p>
    positive target embedding—— $\boldsymbol{c}^{PE}$ <p>
    negative target embedding—— $\boldsymbol{c}^{NE}$
    - $\boldsymbol{c}=\{\boldsymbol{c}^{SOT},\boldsymbol{c}^{PE},\boldsymbol{c}^{NE},\boldsymbol{c}_0^{EOT},\cdots,\boldsymbol{c}_{N-|\boldsymbol{p}|-2}^{EOT}\}$

目的:消除EOT中包含的negative target的信息
- 1.构建嵌入矩阵$\boldsymbol{\chi}:\boldsymbol{\chi}=\left[\boldsymbol{c}^{NE},\boldsymbol{c}_0^{EOT},\cdots,\boldsymbol{c}_{N-|\boldsymbol{p}|-2}^{EOT}\right]$
- 2.SVD分解:$\boldsymbol{\chi}=\mathbf{U}\boldsymbol{\Sigma}\mathbf{V}^T$
    - $\boldsymbol{\Sigma}=diag(\sigma_0,\sigma_1,....,\sigma_{n_0})$
    - $\boldsymbol{\sigma}_0\geq\cdots\geq\boldsymbol{\sigma}_{n_0},n_0=\min(M,N-|\boldsymbol{p}|-1)$
- 3.SWR抑制：SVD分解后的主singular value独赢negative target信息
    - $\hat{\sigma}=e^{-\sigma}*\sigma$'
    - 更新后的嵌入矩阵为$\hat{\boldsymbol{\chi}}=\mathbf{U}\hat{\boldsymbol{\Sigma}}\mathbf{V}^T$,其中$\hat{\boldsymbol{\Sigma}}=diag(\hat{\sigma_0},\hat{\sigma_1},\cdots,\hat{\sigma_{n_0}})$ ,<p>SWR更新后的
嵌入矩阵和text embeddings为$\hat{\boldsymbol{\chi}}=\left[\hat{\boldsymbol{c}}^{NE},\hat{\boldsymbol{c}}_0^{EOT},\cdots,\hat{\boldsymbol{c}}_{N-|\boldsymbol{p}|-2}^{EOT}\right]$和
$\hat{\boldsymbol{c}}=\{\boldsymbol{c}^{SOT},\boldsymbol{c}^{PE},\boldsymbol{\hat{c}}^{NE},\boldsymbol{\hat{c}}_{0}^{EOT},\cdots,\boldsymbol{\hat{c}}_{N-|\boldsymbol{p}|-2}^{EOT}\}.$

核心代码`wo_utils.py`,三个方法
- punish_wight
- woword_eot_context
- woword_reweight

punish_wight根据指定的method和alpha对wo_batch进行修改。
- wo_batch: 一个 PyTorch 张量，代表需要处理的权重或嵌入
- latent_size
- alpha:用于控制惩罚或者修改的强度或者方式
- method:可选 'weight', 'alpha', 'beta', 'delete', 'soft-weight'
    - weigth:wo_batch*alpha
    - alpha/beta/delete/soft-weight <p>
        首先进行SVD正交分解，$wo\_batch=U \Sigma V^T$
        - `alpha` 方法：将前 int(latent_size * alpha) 个奇异值置零。这相当于去除原始矩阵中与这些最大的奇异值对应的成分。
        - `beta` 方法：将从 int(latent_size * alpha) 位置开始到末尾的奇异值置零。这相当于去除原始矩阵中与这些较小的奇异值对应的成分。
        - `delete` 方法：删除从 int(latent_size * alpha) 位置开始到末尾的奇异值，并相应地调整U和V的维度。这是一种更彻底的成分移除。
        - `soft-weight` 方法：使用指数函数来软化奇异值，平滑调整奇异值大小
            - 增强(alpha == -.001) <p>
            s *= (torch.exp(.001 * s) * 1.2)
            - 抑制(alpha != -.001) <p>
            s * = torch.exp(-alpha*s)
    - 使用修改后的成分重构$wo\_batch$张量

| 方法        | 描述         | 数学运算               |
|-------------|--------------|---------------------- |
| weight      | 简单标量乘法 | `embedding *= alpha`   |
| alpha       | 零首分量     | `s[:zero_idx] = 0`     |
| beta        | 零最后分量   | `s[zero_idx:] = 0`     |
| delete      | 完全移除分量 | 截断 U、S、V 矩阵       |
| soft-weight | 指数重加权   | `s *= exp(-alpha * s)` |

![alt text](wo.png)

In [3]:
import torch
import random
from scipy.spatial.distance import cdist
# CALC_SIMILARITY 标志启用了可选的余弦距离计算
CALC_SIMILARITY = False

In [ ]:
def punish_wight(wo_batch, latent_size, alpha, method):
    # 如果是weigth就简单相乘
    if method == 'weight':
        wo_batch *= alpha
    elif method in ['alpha', 'beta', 'delete', 'soft-weight']:
        u, s, vh = torch.linalg.svd(wo_batch)
        u = u[:,:latent_size]
        zero_idx = int(latent_size * alpha)
        if method == 'alpha':
            s[:zero_idx] = 0
        elif method == 'beta':
            s[zero_idx:] = 0
        elif method == 'delete':
            s = s[zero_idx:] if zero_idx < latent_size else torch.zeros(latent_size).to(s.device)
            u = u[:, zero_idx:] if zero_idx < latent_size else u
            vh = vh[zero_idx:, :] if zero_idx < latent_size else vh
        elif method == 'soft-weight':
            if CALC_SIMILARITY:
                _s = s.clone()
                _s[zero_idx:] = 0
                _wo_batch = u @ torch.diag(_s) @ vh
                dist = cdist(wo_batch[:,0].unsqueeze(0).cpu(), _wo_batch[:,0].unsqueeze(0).cpu(), metric='cosine')
                print(f'The distance between the word embedding before and after the punishment: {dist}')
            if alpha == -.001:
                s *= (torch.exp(.001 * s) * 1.2)  # strengthen objects (our Appendix.F)
            else:
                s *= torch.exp(-alpha*s)  # suppression EOT (our main paper)

        wo_batch = u @ torch.diag(s) @ vh
    else:
        raise ValueError('Unsupported method')
    return wo_batch

文本嵌入修改 : `woword_eot_context` 在文本编码器输出进入扩散 U-Net 之前进行修改

In [6]:
def woword_eot_context(context, token_indices, alpha, method, n):
    for i, batch in enumerate(context):
        indices = token_indices + [num for num in range(n-1, 77)]
        wo_batch = batch[indices]
        wo_batch = punish_wight(wo_batch.T, len(indices), alpha, method).T
        batch[indices] = wo_batch
    return context

注意力控制 ：`woword_reweight` 在去噪过程中修改交叉注意力图

In [7]:
def woword_reweight(attn, token_indices, alpha):
    # if attn.shape[1] > 32 ** 2:  # avoid memory overhead
    #     return attn
    latent_size = int(attn.shape[1]**0.5)
    assert latent_size**2 == attn.shape[1]
    for head_attn in attn:
        for indice in token_indices:
            wo_attn = head_attn[:, indice].reshape(latent_size, latent_size)
            wo_attn *= alpha  # same as Reweight of P2P
            head_attn[:, indice] = wo_attn.reshape(latent_size**2)
    return attn

### 推理时优化(INFERENCE-TIME TEXT EMBEDDING OPTIMIZATION)

获取text embeddings中$c$中$\boldsymbol{c}^{PE}$和$\boldsymbol{c}^{NE}$对应的attention maps，即$(A_t^{\boldsymbol{PE}},A_t^{\boldsymbol{NE}})$，同时获取SWR更新后的text embeddings——$\hat{c}$对应的$(\hat{A}_t^{\boldsymbol{PE}},\hat{A}_t^{\boldsymbol{NE}})$
- $\mathcal{L}=\lambda_{pl}\mathcal{L}_{pl}+\lambda_{nl}\mathcal{L}_{nl}$
  - $\mathcal{L}_{pl}=\left\|\hat{\boldsymbol{A}}_{\boldsymbol{t}}^{\boldsymbol{PE}}-\boldsymbol{A}_{\boldsymbol{t}}^{\boldsymbol{PE}}\right\|^2$
  - $\mathcal{L}_{nl}=-\left\|\hat{\boldsymbol{A}}_{\boldsymbol{t}}^{\boldsymbol{NE}}-\boldsymbol{A}_{\boldsymbol{t}}^{\boldsymbol{NE}}\right\|^2$

- `AttentionControl`抽象基类
- `AttentionStore`从不同的 UNet 位置（down_cross、mid_cross、up_cross、down_self、mid_self、up_self）捕获注意力图，并在扩散步骤中累积它们以进行分析和优化
- `EmptyControl`类作为基线，不执行注意力修改
- `SpatialReplace`类提供空间注意力替换功能

#### Loss
Loss Type:            
- `mse`: torch.nn.MSELoss(x,y),
- `cosine`: 1-torch.nn.CosineSimilarity(x,y),
- `mae`: torch.nn.L1Loss(x,y)

1.`calc_retain_loss`:确保对于非 token_indices 中指定的词，原始交叉注意力 (attn) 和被擦除交叉注意力 (attn_erase) 之间的相似度高

In [ ]:
def calc_retain_loss(self, attn, attn_erase):
        loss = .0
        for i in [num for num in range(1, self.prompt_n-1)]:
            if i in self.token_indices:
                continue
            loss += self.retain_loss(attn[:,:,i], attn_erase[:,:,i])
        # print(f'\n retain loss: {loss.item()}, ', end=' ')
        return loss

2.`calc_erase_loss`:确保对于 token_indices 中指定的词,原始交叉注意力 (attn) 和被擦除交叉注意力 (attn_erase) 之间的相似度低

In [1]:
def calc_erase_loss(self, attn, attn_erase):
        loss = .0
        for i in self.token_indices:
            loss += self.erase_loss(attn[:,:,i], attn_erase[:,:,i])
        # print(f'erase loss: {loss.item()}')
        return loss

3.`calc_self_retain_loss`:确保对于非 token_indices 中指定的词，在通过 calc_mask 计算得到的重要空间区域内，原始自注意力 (self_attn) 和被擦除自注意力 (self_attn_erase) 之间的相似度高

In [ ]:
def calc_self_retain_loss(self, self_attn, self_attn_erase, mask):
        loss = .0
        h, w = mask[0].shape
        for i in [num for num in range(1, self.prompt_n-1)]:
            if i in self.token_indices:
                continue
            for j, m in enumerate(mask[i-1].reshape(h*w)):
                if m > 0:
                    loss += self.self_retain_loss(self_attn[:,:,j].view(-1).unsqueeze(0),
                                                  self_attn_erase[:,:,j].view(-1).unsqueeze(0))
        # print(f'self retain loss: {loss.item()}, ', end=' ')
        return loss

4.`calc_self_erase_loss`:确保对于 token_indices 中指定的词，在通过 calc_mask 计算得到的重要空间区域内，原始自注意力 (self_attn) 和被擦除自注意力 (self_attn_erase) 之间的相似度低

In [ ]:
def calc_self_erase_loss(self, self_attn, self_attn_erase, mask):
        loss = .0
        h, w = mask[0].shape
        for i in self.token_indices:
            for j, m in enumerate(mask[i-1].reshape(h*w)):
                if m > 0:
                    loss += self.self_erase_loss(self_attn[:,:,j].view(-1).unsqueeze(0),
                                                 self_attn_erase[:,:,j].view(-1).unsqueeze(0))
        # print(f'self erase loss: {loss.item()}')
        return loss

calc_self_erase_loss和calc_self_retain_loss需要使用calc_mask乘胜空间掩码

forward 函数的整体流程：<p>
接收四种注意力输入：原始交叉注意力 (attn)，被擦除交叉注意力 (attn_erase)，原始自注意力 (self_attn)，被擦除自注意力 (self_attn_erase)。<p>
将所有输入转换为 torch.double 精度进行计算。<p>
如果 lambda_self_retain 或 lambda_self_erase 非零，则调用 calc_mask 计算掩码。<p>
重塑交叉注意力 (attn 和 attn_erase) 以便计算（将空间维度展平）。<p>
根据相应的 lambda 权重，调用四个损失计算函数，并将结果累加到 attn_loss。<p>
返回最终的总损失 loss (即 attn_loss)。

# Code usage